In [1]:
import pandas as pd
import requests
import time
from urllib.parse import urlparse
from pdf2image import convert_from_path
import csv
import os
import shutil
from PIL import Image
import pytesseract 
import pandas as pd
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama
import json
import re

In [2]:
def get_image_from_path(pdf_file_path):
    # ================= CONFIGURATION =================
    #PDF_FOLDER = "all_pdf_files_raw"              # Folder containing your PDFs
    #OUTPUT_FOLDER = "input"  # Where to save first pages
    DPI = 200                              # Image quality
    POPPLER_PATH = r"C:/Users/akela/Downloads/Release-25.07.0-0/poppler-25.07.0/Library/bin"  # Update if needed
    # =================================================
    try:
        # Convert ONLY the first page
        #print(f"Processing: {pdf_file} → extracting first page...")
        pages = convert_from_path(
            pdf_file_path,
            dpi=DPI,
            poppler_path=POPPLER_PATH,
            first_page=1,
            last_page=1  # This ensures only page 1 is converted
        )
        
        # Save the first (and only) page
        if pages:
            #output_name = os.path.splitext(pdf_file_path)[0] + "_first_page.png"
            #output_path = os.path.join(OUTPUT_FOLDER, output_name)
            #pages[0].save(output_path, "PNG")
            #print(f"Saved: {output_path}")
            return pages[0]
        else: 
            print(f"Warning: No pages extracted from {pdf_file_path}")
            return None
    except Exception as e:
        print(f"Failed to process {pdf_file_path}: {e}")
        return None

In [3]:
def update_csv(data,saving_path,field_names):
    #output_name = os.path.splitext(pdf_file_path)[0] + "_first_page.png"
    #output_path = os.path.join(OUTPUT_FOLDER, output_name)
    #img.save(output_path, "PNG")
    #data["src"]=img
    print("_data_row:", data)

    try:
        file_exists = os.path.isfile(saving_path)
        with open(saving_path, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=field_names)
            # If the file didn't exist before opening, write the header first
            
            if not file_exists:
                writer.writeheader()
                print(f"Created new file '{saving_path}' and wrote header.")
            # Write the single row
            writer.writerow(data)
            print(f"Successfully appended a new row to '{saving_path}'.")

    except Exception as e:
        print(f"Error processing {data}: {e}")

In [ ]:
#from create_folders import create_all_necessary_folders
from s0_saving_files_to_folder import get_files
#from engine import process_images

url_name_csv_path = "url_name.csv"
raw_csv = "../CF-AN-equities-24-01-2026-to-25-01-2026.csv"
download_dir = "../pdf_files_unprocessed"

get_files(raw_csv,download_dir, url_name_csv_path)

#create_all_necessary_folders()

#for pdf in os.listdir("all_pdf_files_raw"):
#    pdf_file_path = os.path.join("all_pdf_files_raw", pdf)
#    print("processing file:", pdf_file_path)
#    img = get_image_from_path(pdf_file_path)
#    if img:
#        process_images(img, pdf)
#        try:
#            shutil.move(pdf_file_path, os.path.join("all_pdf_files_raw_processed", pdf))
#            print("File moved successfully!")
#        except FileNotFoundError:
#            print("Error: The source file was not found.")
#        except PermissionError:
#            print("Error: Permission denied. Close the PDF if it's open.")

Total PDF URLs found: 564

[1/564] Trying → https://nsearchives.nseindia.com/corporate/HIGHWAY123_10122025203348_HIL_Reg_30_Greenfield_101225.pdf
Successfully appended a new row to 'url_name.csv'.

[2/564] Trying → https://nsearchives.nseindia.com/corporate/PURVA_10122025183257_seintimationsd.pdf
Successfully appended a new row to 'url_name.csv'.

[3/564] Trying → https://nsearchives.nseindia.com/corporate/RISHABH_10122025173616_RIL__Bagging_Receiving_the_Order_08122025.pdf
Successfully appended a new row to 'url_name.csv'.

[4/564] Trying → https://nsearchives.nseindia.com/corporate/SURYAROSNI_10122025143911_Received_Order_10122025.pdf
Successfully appended a new row to 'url_name.csv'.

[5/564] Trying → https://nsearchives.nseindia.com/corporate/GLOBE2024_10122025143656_Final_Initmation_Tender_.pdf
Successfully appended a new row to 'url_name.csv'.

[6/564] Trying → https://nsearchives.nseindia.com/corporate/DBL_10122025103139_Intimation.pdf
Successfully appended a new row to 'url_nam

In [6]:
from processing_unidentified_photos import process_images

process_images("output_chumma/z_unprocessed_files","output_chumma/ai_segmentation.csv")

Processing 1th file: 12_ZAGGLE_24012025143042_Intimation_Tyger.png


KeyboardInterrupt: 

         SYMBOL                                   COMPANY NAME  \
0    CYBERMEDIA                    Cyber Media (India) Limited   
1       MEDPLUS                Medplus Health Services Limited   
2     POWERGRID        Power Grid Corporation of India Limited   
3     POWERGRID        Power Grid Corporation of India Limited   
4    JPASSOCIAT                  Jaiprakash Associates Limited   
..          ...                                            ...   
192        TEGA                        Tega Industries Limited   
193  ADANIENSOL                 Adani Energy Solutions Limited   
194    ADANIENT                      Adani Enterprises Limited   
195  ADANIPORTS  Adani Ports and Special Economic Zone Limited   
196        ATGL                        Adani Total Gas Limited   

                           SUBJECT  \
0    Copy of Newspaper Publication   
1                  General Updates   
2             Change in Management   
3         Outcome of Board Meeting   
4                

In [ ]:
import pandas as pd
from pathlib import Path

# 1. Define the segment names
segments = [
    "change_in_management", "law_court_govt", "news_papers",
    "investor_presentation", "meeting_update", "esops",
    "outcome_of_boardmeeting", "insolvency", "manufacturing_plat_inspection",
    "transcripts", "credit_rating", "insider_trading",
    "order_update_mou", "cash_flow", "esg_rating",
    "postal_ballot_notices", "re_lodgement", "subsidary_ncbs",
    "amalgamation", "joint_ventures", "subsidary_status_change"
]

def create_empty_segment_csvs():
    # Define the requested headers
    headers = ["segment_name","pdf_name"]
    
    for segment in segments:
        # Construct the filename
        file_name = f"{segment}_unprocessed.csv"
        
        # Create an empty DataFrame with just the headers
        empty_df = pd.DataFrame(columns=headers)
        
        # Save the file
        empty_df.to_csv(file_name, index=False)
        print(f"📁 Created empty file: {file_name}")

# Run the function
create_empty_segment_csvs()

📁 Created empty file: change_in_management_unprocessed.csv
📁 Created empty file: law_court_govt_unprocessed.csv
📁 Created empty file: news_papers_unprocessed.csv
📁 Created empty file: investor_presentation_unprocessed.csv
📁 Created empty file: meeting_update_unprocessed.csv
📁 Created empty file: esops_unprocessed.csv
📁 Created empty file: outcome_of_boardmeeting_unprocessed.csv
📁 Created empty file: insolvency_unprocessed.csv
📁 Created empty file: manufacturing_plat_inspection_unprocessed.csv
📁 Created empty file: transcripts_unprocessed.csv
📁 Created empty file: credit_rating_unprocessed.csv
📁 Created empty file: insider_trading_unprocessed.csv
📁 Created empty file: order_update_mou_unprocessed.csv
📁 Created empty file: cash_flow_unprocessed.csv
📁 Created empty file: esg_rating_unprocessed.csv
📁 Created empty file: postal_ballot_notices_unprocessed.csv
📁 Created empty file: re_lodgement_unprocessed.csv
📁 Created empty file: subsidary_ncbs_unprocessed.csv
📁 Created empty file: amalgama

In [7]:
headers = ["segment_name","pdf_name"]
# Construct the filename
file_name = f"financial_results.csv"
        
        # Create an empty DataFrame with just the headers
empty_df = pd.DataFrame(columns=headers)
        
        # Save the file
empty_df.to_csv(file_name, index=False)
print(f"📁 Created empty file: {file_name}")

📁 Created empty file: financial_results.csv
